In [71]:
import pandas as pd
import numpy as np
# not skipping empty values, to demonstrate data preprocessing steps later
df = pd.read_csv('yellow_tripdata_2015-01.csv', na_filter=False) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12748986 entries, 0 to 12748985
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        int64  
 4   trip_distance          float64
 5   pickup_longitude       float64
 6   pickup_latitude        float64
 7   RateCodeID             int64  
 8   store_and_fwd_flag     object 
 9   dropoff_longitude      float64
 10  dropoff_latitude       float64
 11  payment_type           int64  
 12  fare_amount            float64
 13  extra                  float64
 14  mta_tax                float64
 15  tip_amount             float64
 16  tolls_amount           float64
 17  improvement_surcharge  object 
 18  total_amount           float64
dtypes: float64(11), int64(4), object(4)
memory usage: 1.8+ GB


In [72]:
# Pick necessary columns, drop the rest
df2 = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'total_amount']]
##df2 = df[['pickup_longitude', 'pickup_latitude']]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12748986 entries, 0 to 12748985
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   tpep_pickup_datetime   object 
 1   tpep_dropoff_datetime  object 
 2   passenger_count        int64  
 3   trip_distance          float64
 4   pickup_longitude       float64
 5   pickup_latitude        float64
 6   dropoff_longitude      float64
 7   dropoff_latitude       float64
 8   total_amount           float64
dtypes: float64(6), int64(1), object(2)
memory usage: 875.4+ MB


In [73]:
df2_headers = df2.dtypes.index
##Check the unique value to briefly see if there's noise in the data
for i in df2_headers:
    ##Change the blank between characters into underline "_"
   print( i, ":\n",df2["{0}".format(i)].unique())
   print( i, ":\n",df2["{0}".format(i)].value_counts(),"\n\n")

tpep_pickup_datetime :
 ['2015-01-15 19:05:39' '2015-01-10 20:33:38' '2015-01-10 20:33:39' ...
 '2015-01-10 18:16:49' '2015-01-26 06:45:05' '2015-01-09 15:04:28']
tpep_pickup_datetime :
 2015-01-05 09:39:49    94
2015-01-12 19:02:27    41
2015-01-12 18:55:17    39
2015-01-14 16:13:52    32
2015-01-14 13:20:13    29
                       ..
2015-01-30 03:22:29     1
2015-01-26 20:35:16     1
2015-01-22 06:10:06     1
2015-01-09 07:05:00     1
2015-01-29 04:18:25     1
Name: tpep_pickup_datetime, Length: 2438284, dtype: int64 


tpep_dropoff_datetime :
 ['2015-01-15 19:23:42' '2015-01-10 20:53:28' '2015-01-10 20:43:41' ...
 '2015-01-10 06:38:03' '2015-01-10 06:17:46' '2015-01-26 10:32:57']
tpep_dropoff_datetime :
 2015-01-02 00:00:00    142
2015-01-04 00:00:00    133
2015-02-01 00:00:00    132
2015-01-03 00:00:00    129
2015-01-11 00:00:00    119
                      ... 
2015-01-06 03:06:05      1
2015-01-22 00:35:32      1
2015-01-21 05:26:44      1
2015-01-29 00:22:25      1
2015-01

According to the output, data is clean. No empty value.
Remove points not in New York City.

New York City Borough Boundary Metadata
West -74.257159 East -73.699215
North 40.915568 South 40.495992

In [74]:
##Remove invalid locations

df2 = df2[df2['pickup_longitude'].between(-74.257159, -73.699215)]
df2 = df2[df2['pickup_latitude'].between(40.495992,40.915568)]

In [75]:
##Check again
df2.info()
df2_headers = df2.dtypes.index
##Check the unique value to briefly see if there's noise in the data
for i in df2_headers:
    ##Change the blank between characters into underline "_"
   print( i, ":\n",df2["{0}".format(i)].unique())
   print( i, ":\n",df2["{0}".format(i)].value_counts(),"\n\n")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12503257 entries, 0 to 12748985
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   tpep_pickup_datetime   object 
 1   tpep_dropoff_datetime  object 
 2   passenger_count        int64  
 3   trip_distance          float64
 4   pickup_longitude       float64
 5   pickup_latitude        float64
 6   dropoff_longitude      float64
 7   dropoff_latitude       float64
 8   total_amount           float64
dtypes: float64(6), int64(1), object(2)
memory usage: 953.9+ MB
tpep_pickup_datetime :
 ['2015-01-15 19:05:39' '2015-01-10 20:33:38' '2015-01-10 20:33:39' ...
 '2015-01-10 18:16:49' '2015-01-26 06:45:05' '2015-01-09 15:04:28']
tpep_pickup_datetime :
 2015-01-24 15:15:44    28
2015-01-31 19:47:55    26
2015-01-23 15:15:14    25
2015-01-30 20:46:03    23
2015-01-08 20:24:24    23
                       ..
2015-01-26 03:35:20     1
2015-01-12 06:03:04     1
2015-01-28 01:51:59     1
2015-

In [76]:
##Concatenate Lon and Lat into loc column
##geometry = [{"type":"Point", "coordinates":[x,y]} for x,y in zip(df2.pickup_longitude, df2.pickup_latitude)]
##df2['loc'] = geometry

In [77]:
df2.to_csv('taxi_data_new.csv', index=False)